<a href="https://colab.research.google.com/github/anaperez10/Analiticalll-Finanzas/blob/main/Analitica_3_Seguros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Carga de librerias, paquetes y bases de datos**

In [1]:
#Conexión con colab
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from unicodedata import normalize

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import plotly.graph_objs as go #Graficos
import plotly.express as px
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt ### gráficos
from plotly.subplots import make_subplots

In [3]:
expustos= pd.read_table('/content/drive/MyDrive/CursoAnalitica3/BD_Expuestos.txt')
socio =pd.read_table('/content/drive/MyDrive/CursoAnalitica3/BD_Sociodemograficas.txt', encoding='latin-1')
siniestros =pd.read_table('/content/drive/MyDrive/CursoAnalitica3/DB_Siniestros.txt',encoding='latin-1')

**Limpieza y transformación de datos**

Base de datos de expuestos

In [4]:
expustos.head(5)

,Asegurado_Id,Poliza_Asegurado_Id,FECHA_INICIO,FECHA_CANCELACION,FECHA_FIN
0,16286173,125696117,2018/01/01,2018/04/10,2018/04/10
1,11885152,141320760,2018/01/01,NaN,2019/12/31
2,763957,162532851,2018/01/01,NaN,2019/12/31
3,29082204,154620473,2018/01/01,2018/04/19,2018/04/19
4,4567640,93633862,2018/01/01,NaN,2019/12/31


In [5]:
expustos.columns

Index(['Asegurado_Id', 'Poliza_Asegurado_Id', 'FECHA_INICIO',
       'FECHA_CANCELACION', 'FECHA_FIN'],
      dtype='object')

In [6]:
expustos.columns= expustos.columns.str.lower()

In [7]:
expustos.dtypes

asegurado_id            int64
poliza_asegurado_id     int64
fecha_inicio           object
fecha_cancelacion      object
fecha_fin              object
dtype: object

In [8]:
expustos.shape

(300900, 5)

In [9]:
expustos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300900 entries, 0 to 300899
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   asegurado_id         300900 non-null  int64 
 1   poliza_asegurado_id  300900 non-null  int64 
 2   fecha_inicio         300900 non-null  object
 3   fecha_cancelacion    151963 non-null  object
 4   fecha_fin            300900 non-null  object
dtypes: int64(2), object(3)
memory usage: 11.5+ MB


In [10]:
expustos['fecha_inicio']=pd.to_datetime(expustos['fecha_inicio'])
expustos['fecha_cancelacion']=pd.to_datetime(expustos['fecha_cancelacion'])
expustos['fecha_fin']=pd.to_datetime(expustos['fecha_fin'])

In [11]:
expustos.dtypes

asegurado_id                    int64
poliza_asegurado_id             int64
fecha_inicio           datetime64[ns]
fecha_cancelacion      datetime64[ns]
fecha_fin              datetime64[ns]
dtype: object

In [12]:
expustos=expustos.fillna(0)

In [13]:
def categorizar(fila):
    fecha_cancelacion = fila.fecha_cancelacion
    if fecha_cancelacion  == 0:
      return 0
    else:
      return 1


In [14]:
expustos["Cancelacion"] = expustos.apply(categorizar, axis=1)

In [15]:
expustos

,asegurado_id,poliza_asegurado_id,fecha_inicio,fecha_cancelacion,fecha_fin,Cancelacion
0,16286173,125696117,2018-01-01,2018-04-10 00:00:00,2018-04-10,1
1,11885152,141320760,2018-01-01,0,2019-12-31,0
2,763957,162532851,2018-01-01,0,2019-12-31,0
3,29082204,154620473,2018-01-01,2018-04-19 00:00:00,2018-04-19,1
4,4567640,93633862,2018-01-01,0,2019-12-31,0
...,...,...,...,...,...,...
300895,28476983,53646742,2018-01-01,0,2019-12-31,0
300896,8479396,89963102,2018-01-01,2018-03-08 00:00:00,2018-03-08,1
300897,6598270,165960557,2018-02-28,0,2019-12-31,0
300898,1000907,105811540,2018-01-01,0,2019-12-31,0


Base de datos de siniestros

In [16]:
siniestros.head(5)

,Mes_Pago,Asegurado_Id,Reclamacion,Diagnostico_Codigo,Diagnostico_Desc,Eventos,Valor_Pagado
0,201803,4834968,EXAMENES DE DIAGNOSTICO,Z011,EXAMEN DE OÍDOS Y DE LA AUDICIÓN,1,111600.00
1,201909,14915125,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,1,50468.00
2,201807,42959949,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,1,610452.00
3,201803,30540131,CONSULTA EXTERNA,9,DIAGNÓSTICO PENDIENTE,1,42160.00
4,201905,21442420,LABORATORIO CLINICO,9,DIAGNÓSTICO PENDIENTE,18,617336.48


In [17]:
siniestros.columns

Index(['Mes_Pago', 'Asegurado_Id', 'Reclamacion', 'Diagnostico_Codigo',
       'Diagnostico_Desc', 'Eventos', 'Valor_Pagado'],
      dtype='object')

In [18]:
siniestros.dtypes

Mes_Pago                int64
Asegurado_Id            int64
Reclamacion            object
Diagnostico_Codigo     object
Diagnostico_Desc       object
Eventos                 int64
Valor_Pagado          float64
dtype: object

In [19]:
siniestros.shape

(3308480, 7)

In [20]:
siniestros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3308480 entries, 0 to 3308479
Data columns (total 7 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Mes_Pago            int64  
 1   Asegurado_Id        int64  
 2   Reclamacion         object 
 3   Diagnostico_Codigo  object 
 4   Diagnostico_Desc    object 
 5   Eventos             int64  
 6   Valor_Pagado        float64
dtypes: float64(1), int64(3), object(3)
memory usage: 176.7+ MB


In [21]:
siniestros.isnull().sum()

Mes_Pago              0
Asegurado_Id          0
Reclamacion           0
Diagnostico_Codigo    0
Diagnostico_Desc      0
Eventos               0
Valor_Pagado          0
dtype: int64

In [22]:
siniestros=siniestros.dropna()

In [23]:
#Imprimir categorias de las variables
lista = ['Reclamacion','Diagnostico_Codigo','Diagnostico_Desc']
for i in lista:
    print(siniestros[i].value_counts())
    print('------------------------------------')

CONSULTA EXTERNA                                                   1255793
EXAMENES DE DIAGNOSTICO                                             694238
LABORATORIO CLINICO                                                 520214
CONSULTA DE URGENCIAS                                               365872
CIRUGIA AMBULATORIA POR ENFERMEDAD                                  106397
FISIOTERAPIA ILIMITADA                                               98370
TRATAMIENTOS ESPECIFICOS                                             49708
TRATAMIENTO MEDICO AMBULATORIO POR ACCIDENTES                        39360
CONSULTAS PRENATALES                                                 33685
TRATAMIENTO MEDICO HOSPITALARIO POR ENFERMEDAD                       28504
TRATAMIENTO QUIRURGICO HOSPITALARIO POR ENFERMEDAD                   20484
CIRUGIA AMBULATORIA POR ACCIDENTE                                    17594
TRATAMIENTO AMBULATORIO POR CANCER                                   14949
INFILTRACIONES           

Base de datos sociodemograficas

In [24]:
socio.head(5)

,Asegurado_Id,Sexo_Cd,FechaNacimiento,Regional,CANCER,EPOC,DIABETES,HIPERTENSION,ENF_CARDIOVASCULAR
0,18686469,F,1984/01/17,REGIONAL ANTIOQUIA,0,0,0,0,0
1,16087775,M,1984/03/12,REGIONAL NORTE,0,0,0,0,0
2,35757761,F,1993/12/19,REGIONAL ANTIOQUIA,0,0,0,0,0
3,48691460,M,2000/11/14,REGIONAL ANTIOQUIA,0,0,0,0,0
4,36074099,F,2015/08/06,REGIONAL CENTRO,0,0,0,0,0


In [25]:
socio.columns

Index(['Asegurado_Id', 'Sexo_Cd', 'FechaNacimiento', 'Regional', 'CANCER',
       'EPOC', 'DIABETES', 'HIPERTENSION', 'ENF_CARDIOVASCULAR'],
      dtype='object')

In [26]:
socio.dtypes

Asegurado_Id           int64
Sexo_Cd               object
FechaNacimiento       object
Regional              object
CANCER                 int64
EPOC                   int64
DIABETES               int64
HIPERTENSION           int64
ENF_CARDIOVASCULAR     int64
dtype: object

In [27]:
socio['FechaNacimiento']=pd.to_datetime(socio['FechaNacimiento'])

In [28]:
socio.shape

(267312, 9)

In [29]:
socio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267312 entries, 0 to 267311
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Asegurado_Id        267312 non-null  int64         
 1   Sexo_Cd             267312 non-null  object        
 2   FechaNacimiento     267312 non-null  datetime64[ns]
 3   Regional            267311 non-null  object        
 4   CANCER              267312 non-null  int64         
 5   EPOC                267312 non-null  int64         
 6   DIABETES            267312 non-null  int64         
 7   HIPERTENSION        267312 non-null  int64         
 8   ENF_CARDIOVASCULAR  267312 non-null  int64         
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 18.4+ MB


In [30]:
#Imprimir categorias de las variables
lista = ['Sexo_Cd','Regional','CANCER','EPOC','DIABETES','HIPERTENSION','ENF_CARDIOVASCULAR']
for i in lista:
    print(socio[i].value_counts())
    print('------------------------------------')

F     145430
M     121877
-1         5
Name: Sexo_Cd, dtype: int64
------------------------------------
REGIONAL CENTRO          127149
REGIONAL ANTIOQUIA        49722
REGIONAL OCCIDENTE        46180
REGIONAL NORTE            34845
REGIONAL EJE CAFETERO      9342
Sin Información              73
Name: Regional, dtype: int64
------------------------------------
0    262929
1      4383
Name: CANCER, dtype: int64
------------------------------------
0    266055
1      1257
Name: EPOC, dtype: int64
------------------------------------
0    263774
1      3538
Name: DIABETES, dtype: int64
------------------------------------
0    250566
1     16746
Name: HIPERTENSION, dtype: int64
------------------------------------
0    266387
1       925
Name: ENF_CARDIOVASCULAR, dtype: int64
------------------------------------


In [31]:
#Pasar los nulos de usuarios a sin información
socio['Sexo_Cd'] = socio['Sexo_Cd'].replace('-1','F')

In [32]:
socio=socio.dropna()

In [33]:
socio['edad']=2019-socio.FechaNacimiento.dt.year

In [34]:
socio['edad'].value_counts()

 35     6748
 38     6745
 39     6735
 37     6587
 34     6526
        ... 
-2         3
 102       3
-3         2
 103       2
 105       1
Name: edad, Length: 109, dtype: int64

In [35]:
socio.loc[socio[socio.edad<0].index,['edad']]=0

In [36]:
def categorizar1(fila):
    edad = fila.edad
    if edad  < 6:
      return 'Primera infancia'
    elif edad < 11:
      return 'Infancia'
    elif edad < 18:
      return 'Adolescencia'
    elif edad < 28:
      return 'Jovenes'
    elif edad <59:
      return'Adultez'
    elif edad >60:
      return 'Persona Mayor'


In [37]:
socio["curso de vida"] = socio.apply(categorizar1, axis=1)

Union de las bases de datos

In [38]:
dfcx = pd.merge(socio,siniestros, on = 'Asegurado_Id',how = 'inner')
dfcx.head(3)

,Asegurado_Id,Sexo_Cd,FechaNacimiento,Regional,CANCER,EPOC,DIABETES,HIPERTENSION,ENF_CARDIOVASCULAR,edad,curso de vida,Mes_Pago,Reclamacion,Diagnostico_Codigo,Diagnostico_Desc,Eventos,Valor_Pagado
0,18686469,F,1984-01-17,REGIONAL ANTIOQUIA,0,0,0,0,0,35,Adultez,201912,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,3,1027462.76
1,18686469,F,1984-01-17,REGIONAL ANTIOQUIA,0,0,0,0,0,35,Adultez,201912,CONSULTA EXTERNA,9,DIAGNÓSTICO PENDIENTE,1,67208.00
2,16087775,M,1984-03-12,REGIONAL NORTE,0,0,0,0,0,35,Adultez,201810,CONSULTA DE URGENCIAS,9,DIAGNÓSTICO PENDIENTE,1,83111.00


In [39]:
dfcx.isnull().sum()

Asegurado_Id              0
Sexo_Cd                   0
FechaNacimiento           0
Regional                  0
CANCER                    0
EPOC                      0
DIABETES                  0
HIPERTENSION              0
ENF_CARDIOVASCULAR        0
edad                      0
curso de vida         80806
Mes_Pago                  0
Reclamacion               0
Diagnostico_Codigo        0
Diagnostico_Desc          0
Eventos                   0
Valor_Pagado              0
dtype: int64

In [40]:
expustos1 = expustos.rename(columns={'asegurado_id':'Asegurado_Id'})

In [41]:
dfcx1 = pd.merge(dfcx,expustos1, on = 'Asegurado_Id',how = 'inner')
dfcx1.head(3)

,Asegurado_Id,Sexo_Cd,FechaNacimiento,Regional,CANCER,EPOC,DIABETES,HIPERTENSION,ENF_CARDIOVASCULAR,edad,...,Reclamacion,Diagnostico_Codigo,Diagnostico_Desc,Eventos,Valor_Pagado,poliza_asegurado_id,fecha_inicio,fecha_cancelacion,fecha_fin,Cancelacion
0,18686469,F,1984-01-17,REGIONAL ANTIOQUIA,0,0,0,0,0,35,...,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,3,1027462.76,202863951,2019-10-31,0,2019-12-31,0
1,18686469,F,1984-01-17,REGIONAL ANTIOQUIA,0,0,0,0,0,35,...,CONSULTA EXTERNA,9,DIAGNÓSTICO PENDIENTE,1,67208.00,202863951,2019-10-31,0,2019-12-31,0
2,16087775,M,1984-03-12,REGIONAL NORTE,0,0,0,0,0,35,...,CONSULTA DE URGENCIAS,9,DIAGNÓSTICO PENDIENTE,1,83111.00,139441379,2018-01-01,2019-12-31 00:00:00,2019-12-31,1


In [42]:
dfcx1.shape

(3781424, 22)

In [41]:
dfcx1.isnull().sum()

Asegurado_Id               0
Sexo_Cd                    0
FechaNacimiento            0
Regional                   0
CANCER                     0
EPOC                       0
DIABETES                   0
HIPERTENSION               0
ENF_CARDIOVASCULAR         0
edad                       0
curso de vida          93009
Mes_Pago                   0
Reclamacion                0
Diagnostico_Codigo         0
Diagnostico_Desc           0
Eventos                    0
Valor_Pagado               0
poliza_asegurado_id        0
fecha_inicio               0
fecha_cancelacion          0
fecha_fin                  0
Cancelacion                0
dtype: int64

In [43]:
dfcx1=dfcx1.drop(['curso de vida'], axis=1)

In [44]:
dfcx1["curso de vida"] = dfcx1.apply(categorizar1, axis=1)

In [45]:
dfcx1

,Asegurado_Id,Sexo_Cd,FechaNacimiento,Regional,CANCER,EPOC,DIABETES,HIPERTENSION,ENF_CARDIOVASCULAR,edad,...,Diagnostico_Codigo,Diagnostico_Desc,Eventos,Valor_Pagado,poliza_asegurado_id,fecha_inicio,fecha_cancelacion,fecha_fin,Cancelacion,curso de vida
0,18686469,F,1984-01-17,REGIONAL ANTIOQUIA,0,0,0,0,0,35,...,9,DIAGNÓSTICO PENDIENTE,3,1027462.76,202863951,2019-10-31,0,2019-12-31,0,Adultez
1,18686469,F,1984-01-17,REGIONAL ANTIOQUIA,0,0,0,0,0,35,...,9,DIAGNÓSTICO PENDIENTE,1,67208.00,202863951,2019-10-31,0,2019-12-31,0,Adultez
2,16087775,M,1984-03-12,REGIONAL NORTE,0,0,0,0,0,35,...,9,DIAGNÓSTICO PENDIENTE,1,83111.00,139441379,2018-01-01,2019-12-31 00:00:00,2019-12-31,1,Adultez
3,16087775,M,1984-03-12,REGIONAL NORTE,0,0,0,0,0,35,...,9,DIAGNÓSTICO PENDIENTE,1,194166.64,139441379,2018-01-01,2019-12-31 00:00:00,2019-12-31,1,Adultez
4,16087775,M,1984-03-12,REGIONAL NORTE,0,0,0,0,0,35,...,9,DIAGNÓSTICO PENDIENTE,1,77729.40,139441379,2018-01-01,2019-12-31 00:00:00,2019-12-31,1,Adultez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3781419,4419287,F,1950-10-06,REGIONAL NORTE,0,0,0,0,0,69,...,9,DIAGNÓSTICO PENDIENTE,1,66600.40,55104420,2018-01-01,0,2019-12-31,0,Persona Mayor
3781420,4419287,F,1950-10-06,REGIONAL NORTE,0,0,0,0,0,69,...,H269,"CATARATA, NO ESPECIFICADA",1,5727075.16,55104420,2018-01-01,0,2019-12-31,0,Persona Mayor
3781421,4419287,F,1950-10-06,REGIONAL NORTE,0,0,0,0,0,69,...,9,DIAGNÓSTICO PENDIENTE,24,404240.00,55104420,2018-01-01,0,2019-12-31,0,Persona Mayor
3781422,4419287,F,1950-10-06,REGIONAL NORTE,0,0,0,0,0,69,...,9,DIAGNÓSTICO PENDIENTE,3,835885.24,55104420,2018-01-01,0,2019-12-31,0,Persona Mayor


In [46]:
dfcx1['curso de vida'].value_counts()

Adultez             2097884
Persona Mayor        547524
Primera infancia     386750
Jovenes              301243
Adolescencia         178583
Infancia             176431
Name: curso de vida, dtype: int64

In [47]:
dfcx1=dfcx1.dropna()

**Análisis Exploratorio**

In [48]:
expustos.columns

Index(['asegurado_id', 'poliza_asegurado_id', 'fecha_inicio',
       'fecha_cancelacion', 'fecha_fin', 'Cancelacion'],
      dtype='object')

In [49]:
df=pd.Series(expustos.fecha_fin - expustos.fecha_inicio).dt.days.value_counts().reset_index().rename(columns={'index':'Dias',0:'tamaño'})

In [50]:
#Tiempo en dias en que las personas piden los seguros.
fig=px.bar(df.drop(index=[0,1]),x='Dias', y='tamaño', color_discrete_sequence =  px.colors.sequential.Aggrnyl)
fig.update_layout(font=dict(size=15)) 
fig.show()

In [50]:
#Distribucion de las personas que cancenlan el seguro antes de terminar
# crear dataset
dic = {0:'No canceló',
       1:'Canceló'}
base = expustos.groupby(['Cancelacion'])[['asegurado_id']].count().sort_values('asegurado_id', ascending = False).reset_index()
base['Cancelacion'] = base['Cancelacion'].replace(dic)
# crear gráfica
fig = px.pie(base, values = 'asegurado_id', names ='Cancelacion',
             title= '<b>Participación de las personas que cancenlan el seguro antes de terminar<b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    template = 'simple_white',
    title_x = 0.5)

fig.show()

In [51]:
df=pd.Series(expustos.fecha_inicio.dt.month_name().value_counts()).reset_index().rename(columns={'index':'Mes','fecha_inicio':'tamaño'})

In [52]:
fig=px.bar(df,x='Mes',y='tamaño',color_discrete_sequence =  px.colors.sequential.Aggrnyl )
fig.update_layout(font=dict(size=15))
fig.show()

Exploración de base de datos Siniestros

In [51]:
siniestros.columns

Index(['Mes_Pago', 'Asegurado_Id', 'Reclamacion', 'Diagnostico_Codigo',
       'Diagnostico_Desc', 'Eventos', 'Valor_Pagado'],
      dtype='object')

In [54]:
# crear dataset
base = siniestros.groupby(['Diagnostico_Desc'])[['Valor_Pagado']].sum().sort_values('Valor_Pagado', ascending = False).reset_index().head(10)

# crear gráfica
fig = px.bar(base, x = 'Diagnostico_Desc', y='Valor_Pagado',
             title= '<b>Costos por el diagnostico<b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Año',
    yaxis_title = 'Precio de venta',
    template = 'simple_white',
    title_x = 0.5)

fig.show()

In [55]:
# crear dataset
base = siniestros.groupby(['Reclamacion'])[['Valor_Pagado']].sum().sort_values('Valor_Pagado', ascending = False).reset_index().head(10)

# crear gráfica
fig = px.bar(base, x = 'Reclamacion', y='Valor_Pagado',
             title= '<b>Costos por el tipo de reclamacion<b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Año',
    yaxis_title = 'Precio de venta',
    template = 'simple_white',
    title_x = 0.5)

fig.show()

Exploracion de base de datos Socio 

In [52]:
socio.columns

Index(['Asegurado_Id', 'Sexo_Cd', 'FechaNacimiento', 'Regional', 'CANCER',
       'EPOC', 'DIABETES', 'HIPERTENSION', 'ENF_CARDIOVASCULAR', 'edad',
       'curso de vida'],
      dtype='object')

In [57]:
# crear dataset
base = socio.groupby(['Sexo_Cd'])[['Asegurado_Id']].count().sort_values('Asegurado_Id', ascending = False).reset_index()

# crear gráfica
fig = px.pie(base, values = 'Asegurado_Id', names ='Sexo_Cd',
             title= '<b>Participación del sexo en los seguros de vida b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    template = 'simple_white',
    title_x = 0.5)

fig.show()

In [58]:
# crear dataset
base = socio.groupby(['curso de vida'])[['Asegurado_Id']].count().sort_values('Asegurado_Id', ascending = False).reset_index()

# crear gráfica
fig = px.pie(base, values = 'Asegurado_Id', names ='curso de vida',
             title= '<b>Participación del curso de vida en los seguros de vida b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    template = 'simple_white',
    title_x = 0.5)

fig.show()

In [59]:
# crear dataset
base = socio.groupby(['Regional'])[['Asegurado_Id']].count().sort_values('Asegurado_Id', ascending = False).reset_index().head(10)

# crear gráfica
fig = px.bar(base, x = 'Regional', y='Asegurado_Id',
             title= '<b>Cantidad de personas por region<b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Año',
    yaxis_title = 'Precio de venta',
    template = 'simple_white',
    title_x = 0.5)

fig.show()

Análisis exploratorio de base de datos concatenada

In [53]:
dfcx1.columns

Index(['Asegurado_Id', 'Sexo_Cd', 'FechaNacimiento', 'Regional', 'CANCER',
       'EPOC', 'DIABETES', 'HIPERTENSION', 'ENF_CARDIOVASCULAR', 'edad',
       'Mes_Pago', 'Reclamacion', 'Diagnostico_Codigo', 'Diagnostico_Desc',
       'Eventos', 'Valor_Pagado', 'poliza_asegurado_id', 'fecha_inicio',
       'fecha_cancelacion', 'fecha_fin', 'Cancelacion', 'curso de vida'],
      dtype='object')

In [54]:
dfcx1['año'] = dfcx1['fecha_inicio'].dt.year 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [62]:
# crear dataset

base = dfcx1.groupby(['curso de vida', 'año'])[['Valor_Pagado']].sum().reset_index()
# MSSubClass: Tipo de edificio
# YrSold: Año en que se vende
# SalePrice: Precio de venta

# crear gráfica
fig = px.bar(base, x = 'año', y='Valor_Pagado', color = 'curso de vida', barmode = 'group', title= '<b>costos según el curso de vida por año<b>')

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Año',
    yaxis_title = 'Precio de venta',
    template = 'simple_white',
    title_x = 0.5)

fig.show()

In [63]:
# crear dataset
base = dfcx1.groupby(['Sexo_Cd', 'año'])[['Valor_Pagado']].sum().reset_index()

# crear gráfica
fig = px.bar(base, x = 'año', y='Valor_Pagado', color = 'Sexo_Cd', barmode = 'group', 
             title= '<b>Precio de venta según el tipo de acceso al edificio<b>',
             color_discrete_sequence=px.colors.qualitative.Antique)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Año',
    yaxis_title = 'Precio de venta',
    template = 'simple_white',
    title_x = 0.5)

fig.show()

**Selección de variables**

In [55]:
dfcx1.columns = dfcx1.columns.str.lower()
dfcx1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3688415 entries, 0 to 3781423
Data columns (total 23 columns):
 #   Column               Dtype         
---  ------               -----         
 0   asegurado_id         int64         
 1   sexo_cd              object        
 2   fechanacimiento      datetime64[ns]
 3   regional             object        
 4   cancer               int64         
 5   epoc                 int64         
 6   diabetes             int64         
 7   hipertension         int64         
 8   enf_cardiovascular   int64         
 9   edad                 int64         
 10  mes_pago             int64         
 11  reclamacion          object        
 12  diagnostico_codigo   object        
 13  diagnostico_desc     object        
 14  eventos              int64         
 15  valor_pagado         float64       
 16  poliza_asegurado_id  int64         
 17  fecha_inicio         datetime64[ns]
 18  fecha_cancelacion    object        
 19  fecha_fin            

In [56]:
dfcx1.columns

Index(['asegurado_id', 'sexo_cd', 'fechanacimiento', 'regional', 'cancer',
       'epoc', 'diabetes', 'hipertension', 'enf_cardiovascular', 'edad',
       'mes_pago', 'reclamacion', 'diagnostico_codigo', 'diagnostico_desc',
       'eventos', 'valor_pagado', 'poliza_asegurado_id', 'fecha_inicio',
       'fecha_cancelacion', 'fecha_fin', 'cancelacion', 'curso de vida',
       'año'],
      dtype='object')

In [57]:
#Imprimir categorias de las variables
lista = ['sexo_cd', 'regional', 'cancer',
       'epoc', 'diabetes', 'hipertension', 'enf_cardiovascular','mes_pago', 
       'reclamacion', 'diagnostico_codigo', 'diagnostico_desc',
       'poliza_asegurado_id','cancelacion', 'curso de vida','año']
for i in lista:
    print(dfcx1[i].value_counts())
    print('------------------------------------')

F    2330673
M    1357742
Name: sexo_cd, dtype: int64
------------------------------------
REGIONAL CENTRO          1962074
REGIONAL OCCIDENTE        616629
REGIONAL ANTIOQUIA        585920
REGIONAL NORTE            426668
REGIONAL EJE CAFETERO      96533
Sin Información              591
Name: regional, dtype: int64
------------------------------------
0    3532842
1     155573
Name: cancer, dtype: int64
------------------------------------
0    3646627
1      41788
Name: epoc, dtype: int64
------------------------------------
0    3560404
1     128011
Name: diabetes, dtype: int64
------------------------------------
0    3221948
1     466467
Name: hipertension, dtype: int64
------------------------------------
0    3652662
1      35753
Name: enf_cardiovascular, dtype: int64
------------------------------------
201907    173285
201910    173075
201912    172942
201909    171037
201905    163318
201903    162856
201810    162188
201911    160695
201908    159645
201812    159211
201904 

In [58]:
# Aplicar funciones para homologar a nivel general.
lista = ['sexo_cd', 'regional' ,'reclamacion', 'diagnostico_desc',
        'curso de vida']
for i in lista:
    dfcx1[i] = dfcx1[i].str.lower() #minuscula
    dfcx1[i] = dfcx1[i].str.strip() #espacios
    dfcx1[i] = dfcx1[i].apply(lambda x: normalize("NFKD", x).encode("ascii", "ignore").decode("ascii") if pd.notnull(x) else x)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [59]:
dfcx1.head(5)

,asegurado_id,sexo_cd,fechanacimiento,regional,cancer,epoc,diabetes,hipertension,enf_cardiovascular,edad,...,diagnostico_desc,eventos,valor_pagado,poliza_asegurado_id,fecha_inicio,fecha_cancelacion,fecha_fin,cancelacion,curso de vida,año
0,18686469,f,1984-01-17,regional antioquia,0,0,0,0,0,35,...,diagnostico pendiente,3,1027462.76,202863951,2019-10-31,0,2019-12-31,0,adultez,2019
1,18686469,f,1984-01-17,regional antioquia,0,0,0,0,0,35,...,diagnostico pendiente,1,67208.00,202863951,2019-10-31,0,2019-12-31,0,adultez,2019
2,16087775,m,1984-03-12,regional norte,0,0,0,0,0,35,...,diagnostico pendiente,1,83111.00,139441379,2018-01-01,2019-12-31 00:00:00,2019-12-31,1,adultez,2018
3,16087775,m,1984-03-12,regional norte,0,0,0,0,0,35,...,diagnostico pendiente,1,194166.64,139441379,2018-01-01,2019-12-31 00:00:00,2019-12-31,1,adultez,2018
4,16087775,m,1984-03-12,regional norte,0,0,0,0,0,35,...,diagnostico pendiente,1,77729.40,139441379,2018-01-01,2019-12-31 00:00:00,2019-12-31,1,adultez,2018


In [ ]:
# Convertir variableas categóricas en dummies.
dfcx1_dummies = pd.get_dummies(dfcx1,columns=['sexo_cd','regional', 'reclamacion', 'diagnostico_codigo', 'diagnostico_desc', 'curso de vida'])
dfcx1_dummies.shape